# Tutorial 9: Booleans and Assertions

## PHYS 2600, Spring 2019

## T9.1 - Some Boolean logic

Let's do some basic exercises with using Boolean types and the logical operators `and`, `or`, and `not`.  When working in logic, it's often useful to write down a __truth table__, which lists all possible outcomes of a certain logical operation.  Here are the truth tables for `and` and `or`, for your reference as you're working below:

<img src="https://physicscourses.colorado.edu/phys2600/phys2600_sp19/img/truth-table.png" width=500px />

### Part A

Use Python statements to verify the following more complicated logical statements:

* (T and F) or T --> T
* (F or F) and T --> F
* (F and T) or (F and F) --> F

In [1]:
print((True and False) or True)
print((False or False) and True)
print((False and True) or (False and False))

True
False
False


### Part B

We've seen this flowchart for a recipe for meringue cookies a couple of times now:

<img src="https://physicscourses.colorado.edu/phys2600/phys2600_sp19/img/lec1-meringue.png" width=600px />

The original recipe is meant for humans, but it's not hard to imagine trying to implement this for a computer!  Suppose we are writing code for a "smart oven" which will cook this recipe: we're currently working on the step where we decide whether to take the cookies out of the oven or not (the right half of the diagram.)

__Complete the function `finished_baking()` below__, which should return `True` if either condition in the blue box on the right is satisfied (i.e. if oven_min is greater than 45 or firm_to_touch is true), and `False` otherwise.

In [2]:
def finished_baking(oven_min, firm_to_touch):
    ### BEGIN SOLUTION
    past_oven_time_limit = oven_min > 45
    
    return past_oven_time_limit or firm_to_touch
    ### END SOLUTION
    

This might seem like a silly example to you, but there's no reason this couldn't be real-life working code!  There could be a real timer that sets the variable `oven_min`, and a microcontroller and some sort of probe that sets `firm_to_touch` in a different test routine.

Now in the next cell, __write your own tests__: create three `assert` statements that use the `finished_baking` function and make sure it gives the expected output.

In [3]:
assert finished_baking(60, False) == True
assert finished_baking(35, True) == True
assert finished_baking(45, False) == False

You probably wrote the tests above using the `==` operator, which is normal for a test like this.  But for Boolean data only, __using `==` is almost always redundant.__  For example, suppose we want use an `assert` to make sure that `my_condition` is `True`.  We could write it like this:

In [4]:
my_condition = True
assert my_condition == True

__Try running the cell above__ with `my_condition` set to both `True` and `False`; you'll find that it works perfectly.  But so does _this:_

In [5]:
my_condition = True
assert my_condition

The statement `my_condition == True` is _identical_ to just `my_condition`, because the equality test maps `True` to `True`, and `False` to `False`!

What if we want to test for a condition being _false_?  Again, we could use `==`, but it turns out that the `not` operator does the same thing:

In [6]:
print(True == False)
print(not True)

print(False == False)
print(not False)

False
False
True
True


So we could just write `assert not thing_that_should_be_false`, and again skip using `==`.  

Try to get in the habit of skipping `==` when testing Boolean variables!  It will make your code cleaner, and if you pick good variable names it can be just as clear to read statements like `assert oven_is_preheated` or `assert not oven_is_on_fire`.

__If you used `==` in your `assert` statements above, go back now and rewrite them without `==`.__

## T9.2 - A cautionary tale about `==`

### Part A

Suppose we want to implement a function `is_triple(x,y)` which returns a Boolean result: `True` if `y` is 3 times `x`, and `False` otherwise.  (Don't ask me _why_ we want such a contrived function, it's just an example.)  Here's one simple way to do it:

In [7]:
def is_triple(x, y):
    return y == 3*x

Looks reasonable, at least for the first few tests, but notice that the fourth test is wrong below:

In [8]:
## Testing cell - all should be True

print(is_triple(3.,9.))
print(is_triple(1,3.))
print(is_triple(1/3.,1.))
print(is_triple(1/355.,3/355.))

True
True
True
False


Why do you think `is_triple` doesn't work as expected in the fourth test case?

_Type your answer here using Markdown._

### Part B

Implement a better version of this function, `is_approx_triple(x,y,prec)`, where `prec` should start with a default value of `1e-10`.  This function should check to make sure that y is equal to 3x only up to differences of order `prec`.  Then run the testing cell below and make sure everything prints `True`.

In [9]:
def is_float_triple(x, y, prec=1e-10):
    ## BEGIN SOLUTION
    return abs(y-3*x) < prec
    ## END SOLUTION
    

In [10]:
## Testing cell - all should be True

print(is_float_triple(3.,9.))
print(is_float_triple(1.,3.))
print(is_float_triple(1/64.,3/64.))
print(is_float_triple(1/10.,3/10.))
print(is_float_triple(1/355.,3/355.))

True
True
True
True
True


Now, __make the default value of `prec` smaller__ until you can reproduce the behavior of the original `is_triple()` function.  Thinking back to [our lecture on floating-point numbers](https://physicscourses.colorado.edu/phys2600/phys2600_sp19/lecture/lec04-numbers-binary/), we recall that the truncation error appears around 1 part in $10^{16}$;  __where do you actually see the behavior change?__

_HINT: there is a common bug that you may have introduced in_ `is_float_triple` - _in fact, I introduced it in my first attempt.  If you're not seeing the expected behavior, try setting `prec=0`.  What should that give you, and what do you see?_

In [11]:
is_float_triple(1/355.,3/355.,prec=1e-18)

False

__The average 64-bit truncation error we estimated on tutorial 4 was about $10^{-16}$, but that's the _relative error_; since 3/335 starts around 0.01, we actually need `prec` of about $10^{-18}$ to recover the disagreement.__

__Important info:__ we don't have to do this by hand!  You've probably seen me import the `numpy.testing` module in some test cells already.  There are [a bunch of testing functions available](https://docs.scipy.org/doc/numpy/reference/routines.testing.html) in this module, but the one I commonly use is `numpy.testing.assert_allclose`, which has the following signature:

```
numpy.testing.assert_allclose(actual, desired, rtol=1e-7, atol=0)
```
(and a couple more optional arguments which you will rarely need.)  Here `rtol` is short for "__relative tolerance__", which tests for equality as a fraction of the second argument passed, `desired`.  `atol` is short for "__absolute tolerance__", which is what we did in the example above: it just tests for the difference to be small than `atol`.

To restate it following the documentation: this function carries out the test
```
assert abs(actual - desired) < atol + rtol * abs(desired)
```
Of course, this function has one more advantage over the one we wrote: it works on entire NumPy arrays as well as single numbers.

As the example above illustrated, to avoid things like floating-point truncation error, `rtol` is usually a more appropriate choice!  In general, __only use `atol` if you know to expect some source of absolute error.__

### Part C

Now that you know how to deal with floating-point numbers in comparisons, let's try a more interesting function!  We can always tell whether a given triangle is a _right triangle_ just by knowing the lengths of the three sides: they will satisfy the Pythagorean theorem $a^2 + b^2 = c^2$ if any angle is 90 degrees.

__Implement the function `is_nearly_right_triangle()` below__, which should return `True` if a triangle with sides `s,t,u` is a right triangle, and `False` otherwise.

_(Hint: remember that you don't know beforehand which of `s,t,u` is the longest side...)_

In [12]:
def is_nearly_right_triangle(s,t,u,prec=1e-6):
    ### BEGIN SOLUTION
    # Test all three possibilities for the Pythagorean theorem!
    test_pyth_1 = s**2 + t**2 - u**2 <= prec
    test_pyth_2 = s**2 + u**2 - t**2 <= prec
    test_pyth_3 = t**2 + u**2 - s**2 <= prec
    
    # Return True if any of the three combinations are True
    return test_pyth_1 or test_pyth_2 or test_pyth_3
    ### END SOLUTION
    

In [13]:
import numpy as np

assert not is_nearly_right_triangle(3.0, 3.0, 3.0)
assert is_nearly_right_triangle(3.0, 4.0, 5.0)
assert is_nearly_right_triangle(5.0, 3.0, 4.0)
assert is_nearly_right_triangle(3.0, 5.0, 4.0)
assert is_nearly_right_triangle(np.sqrt(2), 1.0, 1.0)
assert is_nearly_right_triangle(1.4, 1.0, 1.0, prec=0.1)